In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [18]:
import hopsworks
import pandas as pd
from datetime import timedelta
from src.inference import get_feature_store, fetch_predictions

def fetch_hourly_rides(hours):
    current_hour = (pd.Timestamp.now(tz="Etc/UTC") - timedelta(hours=hours)).floor('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour >= current_hour)

    return query.read()

In [19]:
df = fetch_hourly_rides(12)

2025-03-04 08:25:41,329 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 08:25:41,407 INFO: Initializing external client
2025-03-04 08:25:41,410 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 08:25:42,769 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215665
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.48s) 


In [20]:
df

,pickup_hour,pickup_location_id,rides
0,2025-03-03 20:00:00+00:00,86,0
1,2025-03-03 20:00:00+00:00,232,1
2,2025-03-03 20:00:00+00:00,214,0
3,2025-03-03 20:00:00+00:00,77,0
4,2025-03-03 20:00:00+00:00,7,2
...,...,...,...
4262,2025-03-03 20:00:00+00:00,14,0
4263,2025-03-03 20:00:00+00:00,11,0
4264,2025-03-03 20:00:00+00:00,236,182
4265,2025-03-03 20:00:00+00:00,184,0


In [21]:
df_pred = fetch_predictions(12)

2025-03-04 08:25:54,176 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 08:25:54,187 INFO: Initializing external client
2025-03-04 08:25:54,189 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 08:25:55,518 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215665
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.52s) 


In [22]:
df_pred

,pickup_location_id,predicted_demand,pickup_hour
0,28,0.0,2025-03-04 01:00:00+00:00
1,128,0.0,2025-03-04 01:00:00+00:00
2,7,2.0,2025-03-04 01:00:00+00:00
3,130,1.0,2025-03-04 01:00:00+00:00
4,19,0.0,2025-03-04 01:00:00+00:00
...,...,...,...
2003,142,140.0,2025-03-04 14:00:00+00:00
2004,41,6.0,2025-03-04 14:00:00+00:00
2005,226,2.0,2025-03-04 14:00:00+00:00
2006,190,0.0,2025-03-04 14:00:00+00:00


In [23]:
merged_df = pd.merge(df, df_pred, on=['pickup_location_id', 'pickup_hour'])

In [24]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand
0,2025-03-04 01:00:00+00:00,219,0,0.0
1,2025-03-04 01:00:00+00:00,228,0,0.0
2,2025-03-04 01:00:00+00:00,129,1,1.0
3,2025-03-04 01:00:00+00:00,213,0,0.0
4,2025-03-04 01:00:00+00:00,231,2,42.0
...,...,...,...,...
1752,2025-03-04 11:00:00+00:00,195,0,0.0
1753,2025-03-04 11:00:00+00:00,12,2,1.0
1754,2025-03-04 11:00:00+00:00,221,0,0.0
1755,2025-03-04 11:00:00+00:00,144,30,30.0


In [25]:
merged_df['difference'] = merged_df['predicted_demand'] - merged_df['rides']

In [26]:
merged_df.sort_values(["pickup_location_id", "pickup_hour"])

,pickup_hour,pickup_location_id,rides,predicted_demand,difference
210,2025-03-04 01:00:00+00:00,2,0,0.0,0.0
472,2025-03-04 05:00:00+00:00,2,0,0.0,0.0
735,2025-03-04 06:00:00+00:00,2,0,0.0,0.0
886,2025-03-04 07:00:00+00:00,2,0,0.0,0.0
1169,2025-03-04 08:00:00+00:00,2,0,0.0,0.0
...,...,...,...,...,...
696,2025-03-04 06:00:00+00:00,263,89,72.0,-17.0
801,2025-03-04 07:00:00+00:00,263,130,73.0,-57.0
1008,2025-03-04 08:00:00+00:00,263,152,116.0,-36.0
1295,2025-03-04 09:00:00+00:00,263,134,86.0,-48.0


In [27]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand,difference
0,2025-03-04 01:00:00+00:00,219,0,0.0,0.0
1,2025-03-04 01:00:00+00:00,228,0,0.0,0.0
2,2025-03-04 01:00:00+00:00,129,1,1.0,0.0
3,2025-03-04 01:00:00+00:00,213,0,0.0,0.0
4,2025-03-04 01:00:00+00:00,231,2,42.0,40.0
...,...,...,...,...,...
1752,2025-03-04 11:00:00+00:00,195,0,0.0,0.0
1753,2025-03-04 11:00:00+00:00,12,2,1.0,-1.0
1754,2025-03-04 11:00:00+00:00,221,0,0.0,0.0
1755,2025-03-04 11:00:00+00:00,144,30,30.0,0.0


In [28]:
import pandas as pd  
import plotly.express as px
df1 = df
df2 = df_pred

# Merge the DataFrames on 'pickup_location_id' and 'pickup_hour'  
merged_df = pd.merge(df1, df2, on=['pickup_location_id', 'pickup_hour'])  

# Calculate the absolute error  
merged_df['absolute_error'] = abs(merged_df['predicted_demand'] - merged_df['rides'])  

# Group by 'pickup_hour' and calculate the mean absolute error (MAE)  
mae_by_hour = merged_df.groupby('pickup_hour')['absolute_error'].mean().reset_index()  
mae_by_hour.rename(columns={'absolute_error': 'MAE'}, inplace=True)  

# Create a Plotly plot  
fig = px.line(  
    mae_by_hour,  
    x='pickup_hour',  
    y='MAE',  
    title='Mean Absolute Error (MAE) by Pickup Hour',  
    labels={'pickup_hour': 'Pickup Hour', 'MAE': 'Mean Absolute Error'},  
    markers=True  
)  

# Show the plot  
fig.show()

In [29]:
mae_by_hour["MAE"].mean()

12.564598747865677